In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import TypedDict
from dotenv import load_dotenv
load_dotenv()
from pprint import pprint

In [2]:
model = ChatGoogleGenerativeAI(model= "models/gemini-2.0-flash")

In [3]:
class BlogSchema(TypedDict):
    topic : str
    outlines: str
    blog: str

In [4]:
def outline_llm(state: BlogSchema) -> BlogSchema:
    topic = state['topic']
    prompt = PromptTemplate(
        template = 'For the topic {topic} generate detailed outline',
        input_variables = ['topic']
    )
    chain = prompt | model | StrOutputParser()
    outline = chain.invoke({"topic": topic})
    
    return {"outlines": outline}

def blog_llm(state: BlogSchema) -> BlogSchema:
    outlines = state['outlines']
    prompt = PromptTemplate(
        template = 'create a full flegded blog from the outlines \n {outline}',
        input_variables = ['outline']
    )
    chain = prompt | model | StrOutputParser()
    blog = chain.invoke({"outline": outlines})
    
    return {"blog": blog}


In [5]:
graph = StateGraph(state_schema=BlogSchema)

graph.add_node('outline_llm', outline_llm)
graph.add_node('blog_llm', blog_llm)

graph.add_edge(START, 'outline_llm')
graph.add_edge('outline_llm', 'blog_llm')
graph.add_edge('blog_llm', END)

workflow = graph.compile()

In [6]:
final_state = workflow.invoke({"topic": "Data Science", "outlines": "","blog": ""})

In [9]:
print(final_state['outlines'])

## Data Science: Detailed Outline

This outline provides a comprehensive overview of Data Science, covering its core concepts, methodologies, tools, and applications.

**I. Introduction to Data Science**

   A. **What is Data Science?**
      1. Definition and scope of Data Science
      2. Distinguishing Data Science from related fields (e.g., Statistics, Machine Learning, Business Intelligence)
      3. The Data Science process: From data to insights to action
   B. **The Data Science Ecosystem**
      1. Key players: Data Scientists, Data Engineers, Business Analysts, Domain Experts
      2. Roles and responsibilities within a Data Science team
      3. Importance of collaboration and communication
   C. **Applications of Data Science**
      1. Examples across various industries (e.g., healthcare, finance, retail, marketing)
      2. Real-world use cases: Fraud detection, personalized recommendations, predictive maintenance, etc.
   D. **Ethical Considerations in Data Science**
   